# Final form
I would propose using a final form in which there is an explicit convolution and where A and C are defined to be positive. It also does not make much sense to add an arbitary phase to the osciallatory component. More care is also needed to convey that the osciallatory compounent disappears *continuouslsy* at $t<0$.


In [20]:
import numpy as np
from scipy.special import erf
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, FloatSlider
from IPython.display import display

%matplotlib widget

In [21]:
%matplotlib widget

We suggest   
$I(t) = g(t, \Gamma) \ast \{ 1 + A H(-t) - A\exp(-t/t_{mag})  - C(1-\exp(-t/t_{mag})) + D H(t)  \exp(-t/t_{wave})  (\cos(\omega t)-1)$  \}

* $g(t, \Gamma)$ is a Gaussian with width defined in terms of full-width at half-maximum $\Gamma$
* $\ast$ denotes a convolution, which we will do numerically
* $H(t)$ is the heavyside function
* $\exp(-t)$ is an exponential which is set to $1$ for $t<0$.
* The function is always 1 for t<0 for $\Gamma \rightarrow 0$. Since we fit $I_{on}/I_{off}$, it 
* $t_{mag}$ is the magnetic recovery timescale
* $A$ is the drop for $\Gamma \rightarrow 0$. Not the amount the function drops. 
* $C$ is the amount of order that fails to recover 
* The osciallatory component is constructed to be continuous. Either a $1-\cos$ or a $\sin$ form is sensible. An arbitary phase just creates a discontinuity that is (confusingly!) absorbed into $A$. 
* $\omega$ is the frequency of the osciallatory component
* $D$ is the magnitude of the osciallatory component
* $t_{wave}$ is the decay rate of the wave.

In [30]:
def exp(x):
    """Exponential with a maximum of 1"""
    y = np.exp(x)
    y[y>1] = 1
    return y

def H(x):
    return np.heaviside(x, 1)

def decay(t, Gamma=.1, A=.9, C=.2, t_mag=3, D=.5, omega=1/50, t_wave=50):
    """Decay form 
    
    parameters
    ----------
    t : array
        time delay. This must be monotonic
        otherwise there will be problems with the convoution. 
    Gamma: float
        time resolution FWHM
    A : float
        Drop in the Gamma \rightarrow 0 limit 
    C : float
        Amount of the order that fails to recover
    t_mag : float
        recovery timescale 
    D : float
        magnitude of the osciallatory component osciallation
    omega : float
        oscillation frequency
    t_mag : float
        osciallation decay timescale
    """
    dt = np.mean(np.abs(np.diff(t)))
    x = np.arange(-Gamma*5, Gamma*5+dt, dt)
    t_extended = np.arange(t.min() - 5*Gamma, t.max() + 5*Gamma, dt) 
    kernel = np.exp(-(2*np.sqrt(np.log(2))*x/Gamma)**2)
    kernel = kernel / np.sum(kernel)
    y = (1 + A*H(-t_extended)
         - A*exp(-t_extended/t_mag)
         - C*(1-exp(-t_extended/t_mag))
         + H(t_extended)*D*exp(-t_extended/t_wave) * (np.cos(omega*t_extended)-1)
        )
    I_extended = np.convolve(y, kernel, mode='same')
    return np.interp(t, t_extended, I_extended)


def plot_revised_form(Gamma=.1, A=.9, C=.2, t_mag=3, D=.5, omega=1/50, t_wave=50):
    I = decay(t, Gamma=Gamma, A=A, C=C, t_mag=t_mag, D=D, omega=omega, t_wave=t_wave)
    art.set_ydata(I)

fig, ax = plt.subplots()
I = decay(t)
art,  = ax.plot(t, I)
ax.set_xlabel('t')
ax.set_ylabel('I')
ax.set_ylim((-4, 10))

interact(plot_revised_form,
         Gamma=FloatSlider(value=0.15, min=0.00001, max=10, step=0.0001),
         A=FloatSlider(value=.9, min=-1, max=1),
         C=FloatSlider(value=.2, min=-1, max=1),
         D=FloatSlider(value=.2, min=0, max=1),
         omega=FloatSlider(min=0, max=.1, step=0.0001),
         t_mag=FloatSlider(value=.5, min=0.0001, max=5, step=0.0001),
        )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.15, description='Gamma', max=10.0, min=1e-05, step=0.0001), FloatSli…

<function __main__.plot_revised_form(Gamma=0.1, A=0.9, C=0.2, t_mag=3, D=0.5, omega=0.02, t_wave=50)>

For clarify you can look a the osciallatory component

In [31]:
fig, ax = plt.subplots()

def H(x):
    return np.heaviside(x, 1)

def osc(t, omega=.1, D=.2, t_wave=100):
    return H(t)*D*np.exp(-t/t_wave)*(np.cos(omega*t) - 1 )

t = np.linspace(-20, 300, 50000)

I = osc(t)
art, *_  = ax.plot(t, I)

def update(omega=.1, D=.2, t_wave=100):
    I = osc(t, omega=omega, D=D, t_wave=100)
    art.set_ydata(I)

interact(update,
        phi=FloatSlider(value=.001, min=0, max=np.pi, step=0.01))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.1, description='omega', max=0.30000000000000004, min=-0.1), FloatSli…

<function __main__.update(omega=0.1, D=0.2, t_wave=100)>